In [1]:
import sys
import pickle

import numpy as np

sys.path.append('/Users/ikedamorito/Desktop/Gunosy/assignment')
from utils.classifier import NaiveBayes
from utils.train import get_cv_accuracy

## スムージングのパラメータをチューニング

In [2]:
with open('../pkl_objects/processed_data.pkl', 'rb') as f:
    data = pickle.load(f)
np.random.shuffle(data)
    
# グリッドサーチ用のパラメータ
alphas = [1, 0.01, 0.05, 0.1, 0.5]

for alpha in alphas:
    # ナイーブベイズ分類
    nb = NaiveBayes(alpha)   

    # 10-fold cross validationの精度で評価
    get_cv_accuracy(nb, data, K=10)
    print('alpha = {0}\n'.format(alpha))

--- 10-fold cross validation ---
Accuracy: 0.9539083557951482 (+/- 0.06208595169331739)
alpha = 1

--- 10-fold cross validation ---
Accuracy: 0.9609164420485176 (+/- 0.0728162014895481)
alpha = 0.01

--- 10-fold cross validation ---
Accuracy: 0.9597035040431265 (+/- 0.06885143883575867)
alpha = 0.05

--- 10-fold cross validation ---
Accuracy: 0.9590296495956873 (+/- 0.06660682235880278)
alpha = 0.1

--- 10-fold cross validation ---
Accuracy: 0.9567385444743935 (+/- 0.06348209616226524)
alpha = 0.5

